In [1]:
%matplotlib inline
from collections import defaultdict
from scipy.sparse import dok_matrix, csr_matrix
import numpy as np
import time
import math
import random

In [ ]:
def random_limit(distribution):
    val = distribution.random()
    while val > 1 or val < 0:
        val = distribution.random()
    return val

class grader(object):
    def __init__(self, name,bias_mean,bias_tau):
        self.name = name
        self.handins = list()
        self.bias_mean = bias_mean
        self.bias_tau = bias_tau
        
    def add_handin(self, handin):
        self.handins.append(handin)
                
    def grade_handins(self):
        for handin in self.handins:
            B = pm.Normal('B_generator',self.bias_mean,self.bias_tau)
            handin.add_gradeing(self,B.random())

class handin:
    def __init__(self,title,owner,true_value,precision):
        self.title = title
        self.owner = owner
        self.gradeings = dict()
        self.graders = list()
        self.true_val = true_value
        self.precision = precision
    
    def add_grader(self,grader):
        self.graders.append(grader)
    
    def add_gradeing(self,grader,bias):
        obs = pm.Normal('obs_generator',self.true_val+bias,self.precision)
        self.gradeings[grader.name] = random_limit(obs)
        
        
class assignment(object):
    
    def __init__(self, handins_input, graders_input):
        self.handins = dict()
        self.graders = dict()
        for handin in handins_input:
            self.handins[handin.title] = handin
        for grader in graders_input:
            self.graders[grader.name] = grader
    
    def add_handin(self, handin):
        self.handing[handin.title] = handin
        
    def add_grader(self, grader):
        self.graders[grader.title] = grader
    
    def find_ungraded_handin(self, grader):
        
        # sort the handins by the one with the least
        sorted_l = sorted(self.handins.values(),key=lambda x: len(x.graders))
        #i = int(random.uniform(0,len(sorted_l)))
        i = 0
        handin = sorted_l[i]
        while handin in grader.handins or (handin.owner.name == grader.name):
        #while(handin.owner.name == grader.name):
            i += 1
            #i = int(random.uniform(0,len(sorted_l)))
            handin = sorted_l[i]
        return handin
            
    def grade_handins(self,n_gradings):
        self.n_gradings = n_gradings
        # Distribute handins
        for i in xrange(0,n_gradings):
            for grader in self.graders.itervalues():
                h = self.find_ungraded_handin(grader)
                h.add_grader(g)
                grader.add_handin(h)
                
        # grade handins
        for grader in self.graders.itervalues():
            grader.grade_handins()
            
class Course(object):
    
    def __init__(self):
        self.assignments = list()
        self.handins = dict()
        self.graders = dict()
        self.n_gradings = 0
    
    def add_assignment(self,assignment):
        self.assignments.append(assignment)
        for a in self.assignments:
            self.handins.update(a.handins)
            self.graders.update(a.graders)
        self.n_gradings = self.n_gradings + a.n_gradings

In [ ]:
course = Course()

T_mu = pm.Normal('T_mu_generator',0.5,25)
T_tau = pm.Gamma('T_tau_generator',10,0.1)
B_mu = pm.Normal('B_mu_generator',0,100)
B_tau = pm.Gamma('B_tau_generator',50,0.1)

handins_data = list()
graders_data = list()

for i in xrange(0,60):
    g = grader('%i' % i,B_mu.random(),B_tau.random())
    graders_data.append(g)
for i in xrange(0,60):    
    t_mu = random_limit(T_mu)
    h = handin('%i' % i, graders_data[i], t_mu, T_tau.random())
    handins_data.append(h)
    
assignment_data = assignment(handins_data,graders_data)
assignment_data.grade_handins(5)
course.add_assignment(assignment_data)

In [ ]:
handins_data = list()

for i in xrange(60,120):
    t_mu = random_limit(T_mu)
    h = handin('%i' % i, graders_data[i-60], t_mu, T_tau.random())
    handins_data.append(h)
    
assignment_data_2 = assignment(handins_data,graders_data)
assignment_data_2.grade_handins(5)
course.add_assignment(assignment_data_2)

Mock data for testing purpose

In [6]:
from scipy.sparse import dok_matrix

# Dimension
H = 6
G = 6

S = dok_matrix((H, G), dtype=np.float32)

for g in xrange(G):
    for h in xrange(H):    
        if h != g:
            S[h,g] = random.random()

In [7]:
Sh = {}
Sg = {}
cnt = 0

for g in xrange(G):
    Sg[g] = S[:,g]

for h in xrange(H):  
    Sh[h] = S[h,:]    
    cnt += Sh[h].nnz

In [8]:
Sih = {}
Sig = {}

for h in range(H):
    Sih[h] = Sh[h].nonzero()[1]

for g in range(G):
    Sig[g] = Sg[g].nonzero()[1]

In [9]:
# Counts
N_H = S.shape[0] # Number of handins
N_G = S.shape[1] # Number of graders
N_g = 5   # Number of gradings
N_eval = S.getnnz()   # Number of evaluations in total

# Hyperparameters
ga_h = 0.5
la_h = 1.0
al_h = 10.0
be_h = 0.1

ga_g = 0.0
la_g = 1.0
al_g = 50.0
be_g = 0.1

al_e = 10.0
be_e = 1.0
t_h = 500.0
t_g = 100.0

# Prior parameters
u_h = dict()
t_h = dict()
u_g = dict()
t_g = dict()
T = dict()
B = dict()

# Draw from priors
e = np.random.gamma(al_e,1.0/be_e)
t_h = np.random.gamma(np.ones((N_H,1))*al_h,1.0/be_h)
u_h = np.random.normal(ga_h,np.sqrt(1.0/(la_g * t_h)))
T = np.random.normal(u_h,np.sqrt(1.0/t_h)) 
t_g = np.random.gamma(np.ones((N_G,1))*al_g,1.0/be_g)
u_g = np.random.normal(ga_g,np.sqrt(1.0/(la_g * t_g)))
B = np.random.normal(u_g,np.sqrt(1.0/t_g))

In [10]:
# Gibbs sampling

burn_in = 1000  # warm-up steps
samples = 6000 # Gibbs sampling steps

# Tracers initialising
trace_e = np.zeros((samples))
trace_u_h = np.zeros((N_H,samples))
trace_t_h = np.zeros((N_H,samples))
trace_u_g = np.zeros((N_G,samples))
trace_t_g = np.zeros((N_G,samples))
trace_T = np.zeros((N_H,samples))
trace_B = np.zeros((N_G,samples))

In [11]:
def construct_sparse(row_index, data, row_count):
    X = dok_matrix((1,row_count))
    data = np.asarray(data).flatten()
    for ri,d in zip(row_index, data):
        X[0,ri] = d
    return X

In [12]:
tw = time.time()
for r in range(burn_in + samples):
    print "\r%i" % (r+1) + " out of %i" % (burn_in + samples),
    
    # Sample T
    for h in range(H):
        Bh = B[Sh[h].todense().T != 0]
        n_gradings = (Sh[h] > 0.0).sum()
        sum_ = (Sh[h] - construct_sparse(Sih[h],Bh,G)).sum()
        v = e*n_gradings+t_h[h]
        T[h] = np.random.normal((u_h[h]*t_h[h]+e*sum_)/v,np.sqrt(1/v))
        
    # Sample B
    for g in range(G):
        Bg = B.T[Sg[g].todense().T != 0]
        n_gradings = (Sg[g] > 0.0).sum()
        sum_ = (Sg[g] - construct_sparse(Sig[g],Bg,H).T).sum()
        v = e*n_gradings+t_g[g]
        B[g] = np.random.normal((u_g[g]*t_g[h]+e*sum_)/v,np.sqrt(1/v))
        
    sum_ = 0.0
    for h in range(N_H):
        Bh = B[Sh[h].todense().T != 0]
        sum_ = sum_ + np.square((Sh[h] - T[h]+construct_sparse(Sih[h],Bh,G)).sum())
    e = np.random.gamma(al_e+0.5*N_eval,1.0/(be_e+0.5*sum_))
    
    # Sample u_h and t_h
    
    la_ = (la_h+1.0)
    al_ = al_h + 0.5 * la_h + 0.5 * np.square(T-u_h)
    be_ = be_h + 0.5 + 0.5 * 1.0
    t_h = np.random.gamma(al_,1.0/be_)
    u_h = np.random.normal((la_h*ga_h+T)/la_,np.sqrt(1.0/(la_*t_h)))

    # Sample u_g and t_g
   
    la_ = (la_g+1)
    al_ = al_g + 0.5 * la_g + 0.5 * np.square(B-u_g)
    be_ = be_g + 0.5 + 0.5 * 1
    t_g = np.random.gamma(al_,1.0/be_)
    u_g = np.random.normal((la_g*ga_g+B)/la_,np.sqrt(1.0/(t_g)))
    
    # Collect tracings
    if r > burn_in:
        trace_e[r] = e
        trace_u_h[:,r] = u_h.flatten()
        trace_t_h[:,r] = t_h.flatten()
        trace_T[:,r] = T.flatten()
        trace_u_g[:,r] = u_g.flatten()
        trace_t_g[:,r] = t_g.flatten()
        trace_B[:,r] = B.flatten()
print
print "Wall time: %f" % (time.time() - tw)


traces = {'e' : trace_e,
          'u_h' : trace_u_h,
          't_h' : trace_t_h,
          'u_g' : trace_u_g,
          't_g' : trace_t_g,
          'T' : trace_T,
          'B' : trace_B}

598 out of 7000

KeyboardInterrupt: 

In [ ]:
traces['e']

In [ ]:
# Sample u_h and t_h

la_ = (la_h+1.0)
al_ = al_h + 0.5 * la_h + 0.5 * np.square(T-u_h)
be_ = be_h + 0.5 + 0.5 * 1.0
t_h = np.random.gamma(al_,1.0/be_)
u_h = np.random.normal((la_h*ga_h+T)/la_,np.sqrt(1.0/(la_*t_h)))


In [ ]:



    for h in range(N_H):
        handin = data.handins[str(h)]
        n_gradings = len(handin.graders)
        sum_ = 0.0
        for g, val in handin.gradeings.iteritems():
            sum_ = sum_ + val - B[int(g)]
        v = e*n_gradings+t_h[h]
        T[h] = np.random.normal((u_h[h]*t_h[h]+e*sum_)/v,np.sqrt(1/v))

    # Sample B
    for g in range(N_G):
        grader = data.graders[str(g)]
        n_gradings = len(grader.handins)
        sum_ = 0.0
        for h in grader.handins:
            sum_ = sum_ + h.gradeings[str(g)] - T[int(h.title)]
        v = e*n_gradings+t_g[g]
        B[g] = np.random.normal((u_g[g]*t_g[g]+e*sum_)/v,np.sqrt(1/v))

    # Sample e
    sum_ = 0.0
    for h in range(N_H):
        for g, grading in data.handins[str(h)].gradeings.iteritems():
            sum_ = sum_ + np.square(grading - (T[int(h)]+B[int(g)]))
    e = np.random.gamma(al_e+0.5*N_eval,1.0/(be_e+0.5*sum_))

    # Sample u_h and t_h
    for h in range(N_H):
        la_ = (la_h+1.0)
        al_ = al_h + 0.5 * la_h + 0.5 * np.square(T[h]-u_h[h])
        be_ = be_h + 0.5 + 0.5 * 1.0
#            al_ = al_h+0.5
#            be_ = be_h+0.5*((la_h*np.square(T[h]-ga_h))/la_)
        t_h[h] = np.random.gamma(al_,1.0/be_)
        u_h[h] = np.random.normal((la_h*ga_h+T[h])/la_,np.sqrt(1.0/(la_*t_h[h])))

    # Sample u_g and t_g
    for g in range(N_G):
        la_ = (la_g+1)
        al_ = al_g + 0.5 * la_g + 0.5 * np.square(B[g]-u_g[g])
        be_ = be_g + 0.5 + 0.5 * 1
#            al_ = al_g+0.5
#            be_ = be_g+0.5*((la_g*np.square(B[g]-ga_g))/la_)
        t_g[g] = np.random.gamma(al_,1.0/be_)
        u_g[g] = np.random.normal((la_g*ga_g+B[g])/la_,np.sqrt(1.0/(t_g[g])))

    # Collect tracings
    if r > burn_in:
        trace_e.append(e)
        for h in range(N_H):
            trace_u_h[h].append(u_h[h])
            trace_t_h[h].append(t_h[h])
            trace_T[h].append(T[h])
        for g in range(N_G):
            trace_u_g[g].append(u_g[g])
            trace_t_g[g].append(t_g[g])
            trace_B[g].append(B[g])
print
print "Wall time: %f" % (time.time() - tw)


traces = {'e' : trace_e,
          'u_h' : trace_u_h,
          't_h' : trace_t_h,
          'u_g' : trace_u_g,
          't_g' : trace_t_g,
          'T' : trace_T,
          'B' : trace_B}